# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


# Importing Libraries

In [79]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [91]:
# Use this cell for any import statements you add
import shutil
from urllib.parse import urljoin
import nbconvert

---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [81]:
artists = {'CityAlight':"https://www.azlyrics.com/c/cityalight.html",
           'Pat Barrett':"https://www.azlyrics.com/p/patbarrett.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: The scraping of the individual artists pages is allowed based on the robots.txt page as only the following file paths are disallowed: /lyricsdb/ and /song/. The rest are allowed as the page specifies "Allow: /"


In [82]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())

    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'html.parser')
        for link in soup.find_all(href=re.compile("/lyrics/")): 
            # now extract the links to lyrics pages from this page
            lyrics_link = link.get('href')
            # store the links 'lyrics_pages' where the key is the artist and the value is a list of links.
            lyrics_pages[artist].append(lyrics_link)

# References used in this section:
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/
# https://www.geeksforgeeks.org/beautifulsoup-scraping-link-from-html/ 

Let's make sure we have enough lyrics pages to scrape. 

In [83]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [84]:
# Let's see how long it's going to take to pull these lyrics if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For CityAlight we have 45.
The full pull will take for this artist will take 0.12 hours.
For Pat Barrett we have 58.
The full pull will take for this artist will take 0.16 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [85]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)


In [86]:
# Make the lyrics folder here, deleting the old folder if one already exists.

if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [87]:
url_stub = "https://www.azlyrics.com" 
start = time.time()

total_pages = 0 

for artist in lyrics_pages :

    # Build a subfolder for the artist
    artist_folder = os.path.join("lyrics", artist)
    os.makedirs(artist_folder, exist_ok=True)

    # Iterate over the lyrics pages
    for link in lyrics_pages[artist]:
        total_pages += 1
        lyrics_url = urljoin(url_stub, link)
        # Request the lyrics page.
        r_lyrics = requests.get(lyrics_url)
        time.sleep(5 + 10 * random.random())

        # Extract the title and lyrics from the page.
        if r_lyrics.status_code == 200:
            soup_lyrics = BeautifulSoup(r_lyrics.text, 'html.parser')
            title = soup_lyrics.find_all('b')[1].get_text()  # The first <b> tag is artist, 2nd <b> tag is song title, 3rd <b> tag is album
            lyrics = soup_lyrics.find_all('div', class_=None, id=None)[0].get_text(separator='\n') # The lyrics appear in the 1st <div> tag with no class or id

            # Write out the title, two returns ('\n'), and the lyrics. Use `generate_filename_from_url` to generate the filename. 
            filename = generate_filename_from_link(link)
            filepath = os.path.join(artist_folder, filename)

            with open(filepath, 'w', encoding='utf-8') as file:
                file.write(title + '\n\n' + lyrics)

# References used in this section:
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/
# https://stackoverflow.com/questions/10893374/python-confusions-with-urljoin
    

In [88]:
# Find the total run time.
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.35 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [89]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [90]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For Pat Barrett we have 58 files.
For Pat Barrett we have roughly 16442 words, 1255 are unique.
For CityAlight we have 45 files.
For CityAlight we have roughly 11766 words, 923 are unique.
